In [74]:
import os
import matplotlib.pyplot as plt
import shutil
import pandas as pd
import numpy as np

from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
import SimpleITK as sitk

In [2]:
import torch

cuda_version = torch.version.cuda
print("CUDA Version:", cuda_version)


CUDA Version: 11.8


In [75]:
def _flatten(t):
    return [item for sublist in t for item in sublist]


def crop(image, margin, interpolator=sitk.sitkLinear):
    """
    Crops a sitk.Image while retaining correct spacing. Negative margins will lead to zero padding

    Args:
        image:  a sitk.Image
        margin: margins to crop. Single integer or float (percentage crop),
                lists of int/float or nestes lists are supported.
    """
    if isinstance(margin, (list, tuple)):
        assert len(margin) == 3, "expected margin to be of length 3"
    else:
        assert isinstance(margin, (int, float)), "expected margin to be a float value"
        margin = [margin, margin, margin]

    margin = [m if isinstance(m, (tuple, list)) else [m, m] for m in margin]
    old_size = image.GetSize()

    # calculate new origin and new image size
    if all([isinstance(m, float) for m in _flatten(margin)]):
        assert all([m >= 0 and m < 0.5 for m in _flatten(margin)]), "margins must be between 0 and 0.5"
        to_crop = [[int(sz * _m) for _m in m] for sz, m in zip(old_size, margin)]
    elif all([isinstance(m, int) for m in _flatten(margin)]):
        to_crop = margin
    else:
        raise ValueError("Wrong format of margins.")

    new_size = [sz - sum(c) for sz, c in zip(old_size, to_crop)]

    # origin has Index (0,0,0)
    # new origin has Index (to_crop[0][0], to_crop[2][0], to_crop[2][0])
    new_origin = image.TransformIndexToPhysicalPoint([c[0] for c in to_crop])

    # create reference plane to resample image
    ref_image = sitk.Image(new_size, image.GetPixelIDValue())
    ref_image.SetSpacing(image.GetSpacing())
    ref_image.SetOrigin(new_origin)
    ref_image.SetDirection(image.GetDirection())

    return sitk.Resample(image, ref_image, interpolator=interpolator)


# Read and Copy Data

In [3]:
df = pd.read_csv("/home/jaalzate/Tartaglia/Prostate_Tartaglia/New_Dataset/validation_dataset_complete.csv")
groups = df.groupby('subject')


In [90]:
rooth_path = "/mnt/ceib/datalake/FISABIO_datalake/p0042021/"
sub_group = groups.get_group(list(groups.groups.keys())[10])

images_t2 = sub_group[(sub_group["sequence"] == "T2w") & (sub_group['orientation'] == 'ax')].values

In [91]:
img_id = 0
image_t2_path =os.path.join(
        rooth_path, images_t2[img_id][0], images_t2[img_id][1], "mim-mr", images_t2[img_id][2], images_t2[img_id][3]
        )
image_t2_path

'/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000015/ses-000015/mim-mr/anat/sub-000015_ses-000015_acq-fse_run-5_bp-p_vp-ax_T2w.nii.gz'

In [92]:
# Copying img to model predictions folder.
list_of_lists_or_source_folder='Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predict'

image = sitk.ReadImage(image_t2_path)

image = crop(image, margin=[0.2,0.2,0.0])

image_save_path = os.path.join(list_of_lists_or_source_folder, "ProstateSample_0000.nii.gz")

# Save the image and segmentation
sitk.WriteImage(image, image_save_path)

In [88]:
images_list = [join('/home/jaalzate/Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predict','ProstateSample_0000.nii.gz')]
images_outdir = [join('/home/jaalzate/Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_results','ProstateSample.nii.gz')]


In [81]:
images_list

['/home/jaalzate/Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predict/ProstateSample_0000.nii.gz']

In [82]:
# instantiate the nnUNetPredictor
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    perform_everything_on_device=True,
    device=torch.device('cuda', 0),
    verbose=True,
    verbose_preprocessing=True,
    allow_tqdm=True
)

In [83]:
# initializes the network architecture, loads the checkpoint
results_folder = "/nvmescratch/ceib/Prostate/nnUnet/nnUNet_results/Dataset014_ProstateOwn/nnUNetTrainer_100epochs__nnUNetPlans__3d_fullres"
predictor.initialize_from_trained_model_folder(
    results_folder,
    use_folds=(0,1,2,3,4),
    checkpoint_name='checkpoint_best.pth',
)

In [93]:
# Save Segmentations
predictor.predict_from_files([images_list],
                            images_outdir,
                            save_probabilities=False, overwrite=True,
                            num_processes_preprocessing=2, num_processes_segmentation_export=2,
                            folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict
old shape: (36, 308, 308), new_shape: [ 35 257 257], old_spacing: [3.0, 0.3905999958515167, 0.3905999958515167], new_spacing: [3.089737057685852, 0.46875, 0.46875], fn_data: functools.partial(<function resample_data_or_seg_to_shape at 0x7fe3ea11ce50>, is_seg=False, order=3, order_z=0, force_separate_z=None)

Predicting ProstateSample.nii.gz:
perform_everything_on_device: True
Input shape: torch.Size([1, 35, 257, 257])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 8, image size is torch.Size([35, 257, 257]), tile_size [24, 256, 256], tile_step_size 0.5
steps:
[[0, 11], [0, 1], [0, 1]]
move image to device cuda:0
preallocating results arrays on device cuda:0
running prediction


100%|██████████| 8/8 [00:02<00:00,  3.11it/s]


Input shape: torch.Size([1, 35, 257, 257])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 8, image size is torch.Size([35, 257, 257]), tile_size [24, 256, 256], tile_step_size 0.5
steps:
[[0, 11], [0, 1], [0, 1]]
move image to device cuda:0
preallocating results arrays on device cuda:0
running prediction


100%|██████████| 8/8 [00:02<00:00,  3.13it/s]


Input shape: torch.Size([1, 35, 257, 257])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 8, image size is torch.Size([35, 257, 257]), tile_size [24, 256, 256], tile_step_size 0.5
steps:
[[0, 11], [0, 1], [0, 1]]
move image to device cuda:0
preallocating results arrays on device cuda:0
running prediction


...

100%|██████████| 8/8 [00:02<00:00,  3.13it/s]


Input shape: torch.Size([1, 35, 257, 257])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 8, image size is torch.Size([35, 257, 257]), tile_size [24, 256, 256], tile_step_size 0.5
steps:
[[0, 11], [0, 1], [0, 1]]
move image to device cuda:0
preallocating results arrays on device cuda:0
running prediction


100%|██████████| 8/8 [00:02<00:00,  3.11it/s]


Prediction done
sending off prediction to background worker for resampling and export
done with ProstateSample.nii.gz


[None]

In [50]:
# Predict Array
prediction = predictor.predict_from_files([images_list],
                            None,
                            save_probabilities=True, overwrite=True,
                            num_processes_preprocessing=2, num_processes_segmentation_export=2,
                            folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting image of shape torch.Size([1, 23, 341, 339]):
perform_everything_on_device: True


100%|██████████| 4/4 [00:01<00:00,  3.33it/s]


sending off prediction to background worker for resampling

Done with image of shape torch.Size([1, 23, 341, 339]):


# Prediction for Bounding Boxes Data

## Organize Dataset

In [248]:
df_bounding_boxes = pd.read_csv("Prostate_Cancer_TFM/Files/Bounding_Boxes/Only_Bounding.csv")
t2_images = df_bounding_boxes[df_bounding_boxes['modality'] == 'T2'].reset_index(drop=True)

In [249]:
t2_images

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality
0,5586,6040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
1,5586,6040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
2,5596,6050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
3,5700,6173,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/SEG_20240113_211858_507...,AXIAL T2 ALTA RES. (de abajo arriba en próstata),/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
4,5665,6131,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/SEG_20240115_203626_074...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
...,...,...,...,...,...,...,...,...,...,...
226,5636,6097,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/SEG_20240113_202733_764...,T2W_TSE_TRA_IQ,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
227,5613,6070,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/SEG_20240113_203814_089...,AX T2 PROPELLER,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
228,5646,6108,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/SEG_20240113_210319_135...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
229,5399,5819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2


In [250]:
# Create temporal folder to save the images in nnUNet format
tmp_folder_imgs = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_images"
tmp_folder_preds = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predictions"

associated_paths = []

if not os.path.exists(tmp_folder_imgs):
    os.makedirs(tmp_folder_imgs)
if not os.path.exists(tmp_folder_preds):
    os.makedirs(tmp_folder_preds)

# Copying img to model predictions folder.
for i, path in enumerate(t2_images['image_nifti']):

    new_path = os.path.join(tmp_folder_imgs, f"Sample{i}_0000.nii.gz")
    image = sitk.ReadImage(path)

    image = crop(image, margin=[0.2,0.2,0.0])
    associated_paths.append({'original_path': path, 'path_nnunet': new_path})
    sitk.WriteImage(image, new_path)

associated_paths = pd.DataFrame(associated_paths)

In [251]:
associated_paths

,original_path,path_nnunet
0,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
1,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
3,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
4,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
...,...,...
226,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
227,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
228,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
229,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...


## Make Predictions

In [252]:
from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor

In [253]:
folder_images = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_images/"
pred_outdir = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predictions"


In [254]:
# instantiate the nnUNetPredictor
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    perform_everything_on_device=True,
    device=torch.device('cuda', 0),
    verbose=False,
    verbose_preprocessing=False,
    allow_tqdm=True
)

In [255]:
# initializes the network architecture, loads the checkpoint
results_folder = "/nvmescratch/ceib/Prostate/nnUnet/nnUNet_results/Dataset014_ProstateOwn/nnUNetTrainer_100epochs__nnUNetPlans__3d_fullres"
predictor.initialize_from_trained_model_folder(
    results_folder,
    use_folds=(0,1,2,3,4),
    checkpoint_name='checkpoint_best.pth',
)

In [264]:
# Save Segmentations
predictor.predict_from_files(folder_images,
                            pred_outdir,
                            save_probabilities=False, overwrite=False,
                            num_processes_preprocessing=2, num_processes_segmentation_export=2,
                            folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

There are 231 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 231 cases that I would like to predict
overwrite was set to False, so I am only working on cases that haven't been predicted yet. That's 217 cases.

Predicting Sample110:
perform_everything_on_device: True


100%|██████████| 8/8 [00:02<00:00,  3.13it/s]


sending off prediction to background worker for resampling and export
done with Sample110

Predicting Sample111:
perform_everything_on_device: True


100%|██████████| 8/8 [00:02<00:00,  3.13it/s]


sending off prediction to background worker for resampling and export
done with Sample111

Predicting Sample112:
perform_everything_on_device: True


...

100%|██████████| 2/2 [00:00<00:00,  3.79it/s]


sending off prediction to background worker for resampling and export
done with Sample98

Predicting Sample99:
perform_everything_on_device: True


100%|██████████| 8/8 [00:02<00:00,  3.13it/s]


sending off prediction to background worker for resampling and export
done with Sample99


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [282]:
# Move predicted masks to the original folder with the name of the original image
directory = '/nvmescratch/ceib/Prostate/Bounding_Boxes/p0052021/Prostate_Segmentation_AI'
pred_outdir = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predictions"

associated_paths['Path_Segmentation'] = [os.path.join(directory, *path.split('/')[-5:]) for path in associated_paths['original_path']]


for i, row in associated_paths.iterrows():

    path_prediction = os.path.join(pred_outdir, f"Sample{i}.nii.gz")

    new_path = row['Path_Segmentation']

    # Extract the directory part of the new path
    new_directory = os.path.dirname(new_path)

    # Create the new directory if it doesn't exist
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)
    
    shutil.copy(path_prediction,new_path)

In [278]:
# Remove temporal folders
shutil.rmtree(tmp_folder_imgs)
shutil.rmtree(tmp_folder_preds)



In [283]:
# Associate df_bounding_boxes with the predictions

df_merged = pd.merge(df_bounding_boxes, associated_paths, left_on='image_nifti', right_on='original_path', how='left')
df_merged = df_merged.drop(columns=['original_path', 'path_nnunet'])


In [284]:
df_merged

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality,Path_Segmentation
0,5586,6040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
1,5586,6040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
2,5596,6050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
3,5700,6173,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/SEG_20240113_211959_757...,AXIAL DW 1500 PROSTATA,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
4,5700,6173,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/SEG_20240113_211927_274...,AXIAL DW 1000 PROSTATA,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
...,...,...,...,...,...,...,...,...,...,...,...
530,5646,6108,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/SEG_20240113_210039_451...,AXI PELVIS DWI B50-800,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
531,5646,6108,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/SEG_20240113_205921_383...,ADC (10^-6 mm²/s):Nov 26 2019 16-59-11 CET,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,ADC,NaN
532,5646,6108,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/SEG_20240113_205854_334...,DW_Synthetic: Ax FOCUS B-50-800 Synthetic B-2000,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
533,5399,5819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...


In [285]:
df_merged.to_csv("Prostate_Cancer_TFM/Files/Bounding_Boxes/Only_Bounding.csv", index=False)